# AWS S3 Operations using Boto3

In this notebook, we will perform several operations on AWS S3 using Boto3. 

These operations include:

- creating an S3 bucket
- writing an object
- listing objects
- reading the contents of an object
- deleting an object
- updating an object
- and handling errors for non-existent objects. 
  
We will also demonstrate how to write files with different prefixes and query files by prefix.

## For this notebook to run...

You will need to have these libraries installed

- `boto3`
- `boto3-stubs[s3]`
- `rich`
- `ipykernel`

The recommended approach is to set up your `pyproject.toml` like so:

```toml
[project]
...
dependencies = ['importlib-metadata; python_version<"3.8"', "boto3"]

[project.optional-dependencies]
stubs = ["boto3-stubs[s3]"]
notebooks = ["jupyterlab", "ipykernel", "rich"]
...
dev = ["cloud-course-project[test,release,static-code-qa,stubs,notebooks]"]
```

So that `pip install --editable './[dev]'` will install all the necessary dependencies into your venv.

## Instructions for setting up autocompletion in Jupyter Notebooks in VS Code

1. Install the development dependencies:
```sh
pip install --editable './[dev]'
```
2. Select the notebook kernel and point it to your virtual environment:
```sh
which python
```
3. Select the VS Code Python interpreter and point it to your virtual environment:
```sh
which python
```
4. Reload the VS Code window (`Ctrl/Cmd + Shift + P` > `Developer: Reload Window`)


In [1]:
import sys
print(sys.executable)


/home/socratic617/ml_ops_cloud_2024/venv/bin/python


In [2]:
# Import necessary libraries
import boto3
from botocore.exceptions import ClientError
from uuid import uuid4  # randomly generated string
from rich import print  # pretty printing
import os
from typing import Optional

try:
    from mypy_boto3_s3 import S3Client
except ImportError:
    print("mypy_boto3_s3 not installed")

In [3]:
# define constants

# Set the profile and region for the AWS SDK (boto3) to use
os.environ["AWS_PROFILE"] = "cloud-course"
os.environ["AWS_REGION"] = "us-east-2" 

# Access the AWS_REGION variable
aws_region = os.environ.get("AWS_REGION", "us-east-2")
print(f"AWS_REGION: {aws_region}")

# Create a session using the specified profile and region
S3_CLIENT: "S3Client" = boto3.client("s3")

BUCKET_NAME = f"cloud-course-bucket-{str(uuid4())[:4]}"

# Single example object
EXAMPLE_OBJECT_KEY = "example/object/file.txt"
EXAMPLE_OBJECT_CONTENT = "This is a test object."

# Multiple example objects
EXAMPLE_OBJECTS = [
    ("example-a/object/file1.txt", "This is a test object."),
    ("example-a/object/file2.txt", "This is another test object."),
    ("example-a/object/file3.txt", "This is yet another test object."),
    ("example-b/object/file1.txt", "This is a test object."),
    ("example-b/object/file2.txt", "This is another test object."),
    ("example-b/object/file3.txt", "This is yet another test object."),
]

print(f"{BUCKET_NAME=}")
print(f"{EXAMPLE_OBJECT_KEY=}")
print(f"{EXAMPLE_OBJECT_CONTENT=}")

AWS_REGION: us-east-2

BUCKET_NAME='cloud-course-bucket-af1f'

EXAMPLE_OBJECT_KEY='example/object/file.txt'

EXAMPLE_OBJECT_CONTENT='This is a test object.'

## Create a bucket

Here, we create an S3 bucket. Anything you can do in the AWS console, you can do programatically!

In [4]:
try:
    from mypy_boto3_s3.type_defs import CreateBucketOutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def create_bucket(bucket_name: str) -> Optional["CreateBucketOutputTypeDef"]:
    """
    Create an S3 bucket.

    :param bucket_name: Name of the bucket to create
    :type bucket_name: str
    """
    response = S3_CLIENT.create_bucket(
        Bucket=bucket_name
    )
    return response


# Create the bucket
response = create_bucket(bucket_name=BUCKET_NAME)
print(f"Bucket '{BUCKET_NAME}' created successfully.")
print(response)

Bucket 'cloud-course-bucket-af1f' created successfully.

{
    'ResponseMetadata': {
        'RequestId': 'KB7KWPVAMJKE4539',
        'HostId': '0JzxUu4nLM8Uaa3jtfRKvxVsofRPNLrpjBxCkwF5AmnZDnCjRYAvrE53ehuJqElaUJqVfQ9Mtp8=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': '0JzxUu4nLM8Uaa3jtfRKvxVsofRPNLrpjBxCkwF5AmnZDnCjRYAvrE53ehuJqElaUJqVfQ9Mtp8=',
            'x-amz-request-id': 'KB7KWPVAMJKE4539',
            'date': 'Fri, 27 Sep 2024 02:20:03 GMT',
            'location': '/cloud-course-bucket-af1f',
            'server': 'AmazonS3',
            'content-length': '0'
        },
        'RetryAttempts': 0
    },
    'Location': '/cloud-course-bucket-af1f'
}

## Write an object to the bucket

In this exercise, you will write an object to the S3 bucket.

"S3 Paths" are URLs of the form `s3://bucket-name/key`. The `key` is the path to the object in the bucket. For example, `s3://my-bucket-name/my-key` refers to the object with key `my-key` in the bucket `my-bucket-name`.

Some examples:

| Path | Bucket | Object Key |
| --- | --- | --- |
| `s3://my-bucket/images/profile.jpeg` | `my-bucket` | `images/profile.jpeg` |
| `s3://my-bucket/data/2021/01/01/data.csv` | `my-bucket` | `data/2021/01/01/data.csv` |
| `s3://my-bucket/file.json` | `my-bucket` | `file.json` |


In [5]:
try:
    from mypy_boto3_s3.type_defs import PutObjectOutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def write_text_object_to_s3(
    bucket_name: str,
    object_key: str,
    object_content: str,
) -> Optional["PutObjectOutputTypeDef"]:
    """
    Write an object to an S3 bucket.

    :param bucket_name: Name of the bucket to write to
    :param object_key: Key of the object to write
    :param object_content: Content of the object to write
    :return: Response from the put_object call
    """
    response = S3_CLIENT.put_object(Bucket=bucket_name, Key=object_key, Body=object_content)
    return response


# Write the single example to S3
response = write_text_object_to_s3(
    bucket_name=BUCKET_NAME,
    object_key=EXAMPLE_OBJECT_KEY,
    object_content=EXAMPLE_OBJECT_CONTENT,
)
print(response)

# Write the rest of the examples to S3
for object_key, object_content in EXAMPLE_OBJECTS:
    print(f"Writing object to path 's3://{BUCKET_NAME}/{object_key}'")
    write_text_object_to_s3(
        bucket_name=BUCKET_NAME,
        object_key=object_key,
        object_content=object_content,
    )

{
    'ResponseMetadata': {
        'RequestId': '043Y2Z8SGA5PNZ42',
        'HostId': '81sFTu7rN/qPG8ofsc3ZmrKXT1gK1xs6fBa3enelQ2jxONtP6/JT936Sz8QBIypaecfMuvL+l4w=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': '81sFTu7rN/qPG8ofsc3ZmrKXT1gK1xs6fBa3enelQ2jxONtP6/JT936Sz8QBIypaecfMuvL+l4w=',
            'x-amz-request-id': '043Y2Z8SGA5PNZ42',
            'date': 'Fri, 27 Sep 2024 02:20:52 GMT',
            'x-amz-server-side-encryption': 'AES256',
            'etag': '"fac2cbcd7b7417c0325922b689019c65"',
            'server': 'AmazonS3',
            'content-length': '0'
        },
        'RetryAttempts': 0
    },
    'ETag': '"fac2cbcd7b7417c0325922b689019c65"',
    'ServerSideEncryption': 'AES256'
}

Writing object to path 's3://cloud-course-bucket-af1f/example-a/object/file1.txt'

Writing object to path 's3://cloud-course-bucket-af1f/example-a/object/file2.txt'

Writing object to path 's3://cloud-course-bucket-af1f/example-a/object/file3.txt'

Writing object to path 's3://cloud-course-bucket-af1f/example-b/object/file1.txt'

Writing object to path 's3://cloud-course-bucket-af1f/example-b/object/file2.txt'

Writing object to path 's3://cloud-course-bucket-af1f/example-b/object/file3.txt'

## Read the content of an object

In this exercise, you will read the content of an object from the S3 bucket.

In [6]:
from botocore.response import StreamingBody


def read_text_object_from_s3(bucket_name: str, object_key: str, verbose: bool = False) -> str | None:
    """
    Read the content of an object from an S3 bucket.

    :param bucket_name: Name of the bucket to read from
    :param object_key: Key of the object to read
    :return: Content of the object
    """
    response = S3_CLIENT.get_object(Bucket=bucket_name, Key=object_key)

    # Note, we need to read the bytestream from the blob and choose how we wish
    # to interpret the bytes. In this case, we interpret them as utf-8 encoded text.
    content_streaming_body: StreamingBody = response["Body"]
    content: str = content_streaming_body.read().decode("utf-8")
    return content


# Read the content of the example object
content = read_text_object_from_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
if content:
    print(f"Content of object '{EXAMPLE_OBJECT_KEY}':\n'{content}'")

Content of object 'example/object/file.txt':
'This is a test object.'

## Exercise #1 - List objects in the bucket

What if there were more than 1,000 objects in the bucket? How would you list all of them?

Hints
- Look into ["boto3 paginators"](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/paginators.html#creating-paginators)
- OR Look into the [boto3 "resource API"](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/resources.html#resources), a powerful, but potentially less performant, object-oriented way
  of interacting with AWS resources via boto3
- OR consider using "Continuation Tokens" with the `list_objects_v2` method.

In [7]:
try:
    from mypy_boto3_s3.type_defs import ListObjectsV2OutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def list_all_object_keys_in_bucket(bucket_name: str) -> list[str]:
    """
    List all objects in an S3 bucket.

    :param bucket_name: Name of the bucket to list objects from
    :return: List of object keys
    """
    # fill this out...
    import boto3

    # Step 1: Create an S3 client
    client = boto3.client('s3', region_name='us-east-1')

    # Step 2: Create a paginator for the list_objects_v2 operation
    paginator = client.get_paginator('list_objects_v2')

    # Step 3: Use the paginator to iterate over all objects in the specified bucket
    page_iterator = paginator.paginate(Bucket='cloud-course-bucket-af1f')

    # Step 4: Loop through each page of results and print object details
    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                print(obj['Key'])  # Prints the object key (filename)

# List all objects in the bucket
objects = list_all_object_keys_in_bucket(BUCKET_NAME)
if objects:
    print("Objects in bucket:")
    for obj in objects:
        print(f" - {obj}")

example-a/object/file1.txt

example-a/object/file2.txt

example-a/object/file3.txt

example-b/object/file1.txt

example-b/object/file2.txt

example-b/object/file3.txt

example/object/file.txt

## Exercise #2 - Update the object (overwrite with new content)

In this exercise, you will update the content of an existing object in the S3 bucket.

In [8]:
new_content = "This is updated content."

# the "PUT object" command is an upsert, so it will overwrite existing files
response = write_text_object_to_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY, new_content)
print(response)

# Read the updated content of the object
updated_content = read_text_object_from_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
if updated_content:
    print(f"Content of updated object '{EXAMPLE_OBJECT_KEY}':\n{updated_content}")

{
    'ResponseMetadata': {
        'RequestId': '9YV90P95MM259ZKY',
        'HostId': 'yNmJxiLBPuHK/irWEaaYMDlhRs6WK0KFU4onsVtsG52/4iBbOq0RnBztbsuHk44xaZe1Wq73jrg=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'yNmJxiLBPuHK/irWEaaYMDlhRs6WK0KFU4onsVtsG52/4iBbOq0RnBztbsuHk44xaZe1Wq73jrg=',
            'x-amz-request-id': '9YV90P95MM259ZKY',
            'date': 'Fri, 27 Sep 2024 03:01:04 GMT',
            'x-amz-server-side-encryption': 'AES256',
            'etag': '"c5a01ee632fac2f88da9e0aa5a3d8cdd"',
            'server': 'AmazonS3',
            'content-length': '0'
        },
        'RetryAttempts': 0
    },
    'ETag': '"c5a01ee632fac2f88da9e0aa5a3d8cdd"',
    'ServerSideEncryption': 'AES256'
}

Content of updated object 'example/object/file.txt':
This is updated content.

## Exercise #3 - Delete the object from the bucket

In this exercise, you will delete an object from the S3 bucket.

In [9]:
try:
    from mypy_boto3_s3.type_defs import DeleteObjectOutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def delete_object_from_s3(
    bucket_name: str,
    object_key: str,
) -> Optional["DeleteObjectOutputTypeDef"]:
    """
    Delete an object from an S3 bucket.

    :param bucket_name: Name of the bucket to delete the object from
    :param object_key: Key of the object to delete
    :return: Response from the delete_object call
    """

# Step 1: Create an S3 client
client = boto3.client('s3', region_name='us-east-1')  # Specify your AWS region

# Step 2: Specify your bucket name and the object key you want to delete
bucket_name = 'cloud-course-bucket-af1f'  
object_key = 'example-a/object/file1.txt'  

# Step 3: Delete the object
response = client.delete_object(Bucket=bucket_name, Key=object_key)

# Step 4: Print the response to confirm deletion
print(f"Deleted object {object_key} from bucket {bucket_name}. Response: {response}")


# Delete the example object
response = delete_object_from_s3(bucket_name=BUCKET_NAME, object_key=EXAMPLE_OBJECT_KEY)
print(response)



Deleted object example-a/object/file1.txt from bucket cloud-course-bucket-af1f. Response: {'ResponseMetadata': 
{'RequestId': 'G11VDKKQ9FJWWXV1', 'HostId': 
'gAhrNC98tZ2giFrIQTcVm5JIXKgsWsuVE2451Jkuav51ER5xeMiI3j8cl5IFktDtUkU+FjfzAMY=', 'HTTPStatusCode': 204, 
'HTTPHeaders': {'x-amz-id-2': 'gAhrNC98tZ2giFrIQTcVm5JIXKgsWsuVE2451Jkuav51ER5xeMiI3j8cl5IFktDtUkU+FjfzAMY=', 
'x-amz-request-id': 'G11VDKKQ9FJWWXV1', 'date': 'Fri, 27 Sep 2024 03:07:11 GMT', 'server': 'AmazonS3'}, 
'RetryAttempts': 0}}

None

## Exercise #4 - Try to read a non-existent object

In this exercise, you will attempt to read a non-existent object from the S3 bucket. 

Catch, suppress, and print the `ClientError` exception that is raised when you try to read a non-existent object.

In [10]:
print(f"Trying to read the deleted object at 's3://cloud-course-bucket-af1f/example-a/object/file1.txt' ...")

# Delete the object first
delete_object_from_s3(bucket_name=BUCKET_NAME, object_key=EXAMPLE_OBJECT_KEY)

try:
    # Try to read the deleted object
    read_text_object_from_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
except ClientError as err:
    if "NoSuchKey" in str(err):
        print(f"Object '{EXAMPLE_OBJECT_KEY}' not found, as expected after deletion.")
    else:
        print(f"An unexpected error occurred: {err}")

Trying to read the deleted object at 's3://cloud-course-bucket-af1f/example-a/object/file1.txt' ...

## Exercise #5 - Try to delete a non-existent object

**Note:** the result of calling `s3_client.delete_object` on a non-existent object is not what you might expect. It succeeds whether or not there exists an object with the given key.

**Note:** The HTTP status code `204` means `No Content`. Or in other words, the request is successful
but there was nothing to delete.

In [18]:
non_existant_object_key = EXAMPLE_OBJECT_KEY + "_non_existent"

# delete the non-existent object
# ...
import boto3

# Step 1: Create an S3 client
s3_client = boto3.client('s3', region_name='us-east-1') 

# Step 2: Define the bucket name and a non-existent object key
bucket_name = 'cloud-course-bucket-af1f'
non_existent_object_key = 'hotwheelz'  # Non-existent key

# Step 3: Attempt to delete the non-existent object
response = s3_client.delete_object(
  Bucket=bucket_name, 
  Key=non_existent_object_key
)

print(response)

{
    'ResponseMetadata': {
        'RequestId': 'WYCVFWR5RXZRQ9ZN',
        'HostId': 
'hLFXVyZrLtFqwZcc4+m6V5LEtW2xWudHU9oHS/aD4y17D304I0uAcog7lgqomLfcalEar4UEK3AUi05wG2BI7VCyYYQXuLgIFca01E3iFt8=',
        'HTTPStatusCode': 204,
        'HTTPHeaders': {
            'x-amz-id-2': 
'hLFXVyZrLtFqwZcc4+m6V5LEtW2xWudHU9oHS/aD4y17D304I0uAcog7lgqomLfcalEar4UEK3AUi05wG2BI7VCyYYQXuLgIFca01E3iFt8=',
            'x-amz-request-id': 'WYCVFWR5RXZRQ9ZN',
            'date': 'Fri, 27 Sep 2024 03:30:11 GMT',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    }
}

## Exercise #6 - Error handling when deleting an object

In this exercise, you will implement error handling to raise a `FileNotFoundError` if you try to delete a non-existent object, i.e., one that has already been deleted or was never written.

Hint, the `s3_client.head_object(...)` method raises an error with status code `404 - File Not Found`
if no file exists for the given object key.

In [19]:
import boto3
from botocore.exceptions import ClientError

HTTP_FILE_NOT_FOUND_ERROR_CODE = "404"


class S3FileNotFoundError(Exception):
    """Raise this exception when an object at a given path is not found in S3."""


def delete_object_or_error_if_not_exists(bucket_name: str, object_key: str) -> None:
    """
    Delete an object from an S3 bucket with error handling for non-existent objects.

    :param bucket_name: Name of the bucket to delete the object from
    :param object_key: Key of the object to delete

    :raises S3FileNotFoundError: if no object exists at the given path
    :raises ClientError: if an unexpected error occurs when using S3 that is not due to file not found
    """
    s3_client = boto3.client('s3', region_name='us-east-1')
  
    try:
        # Check if the object exists
        s3_client.head_object(Bucket=bucket_name, Key=object_key)
    except ClientError as e:
        # Check for 404 error
        if e.response['Error']['Code'] == "404":
            raise S3FileNotFoundError(f"Object '{object_key}' does not exist in bucket '{bucket_name}'.")
        else:
            # Raise any other ClientError
            raise

    # If the object exists, delete it
    s3_client.delete_object(Bucket=bucket_name, Key=object_key)
    print(f"Object '{object_key}' deleted from bucket '{bucket_name}'.")

# Example usage
BUCKET_NAME = 'cloud-course-bucket-af1f'
EXAMPLE_OBJECT_KEY = 'hotwheelz' 

# Try to delete the non-existent example object with error handling
try:
    delete_object_or_error_if_not_exists(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
except S3FileNotFoundError as err:
    print(err)
except ClientError as e:
    print(f"An unexpected error occurred: {e}")

Object 'hotwheelz' does not exist in bucket 'cloud-course-bucket-af1f'.

## Exercise #7 - List objects in the bucket to confirm they were deleted in the previous exercises

In this exercise, you will list all objects in the S3 bucket to confirm that the object has been deleted.

In [20]:
import boto3

s3_client = boto3.client('s3', region_name='us-east-1')  
BUCKET_NAME = 'cloud-course-bucket-af1f' 

def list_all_object_keys_in_bucket(bucket_name: str):
    """
    List all object keys in the specified S3 bucket.

    :param bucket_name: Name of the S3 bucket
    :return: A list of object keys in the bucket
    """
    object_keys = []
    
    # Use paginator to handle large number of objects
    paginator = s3_client.get_paginator('list_objects_v2')
    
    for page in paginator.paginate(Bucket=bucket_name):
        if 'Contents' in page:  # Check if the 'Contents' key is in the page
            for obj in page['Contents']:
                object_keys.append(obj['Key'])  # Add the object's key to the list
                
    return object_keys

# List all objects in the bucket
objects = list_all_object_keys_in_bucket(BUCKET_NAME)

if objects:
    print("Objects in bucket:")
    for obj in objects:
        print(f" - {obj}")
else:
    print("Bucket is empty.")


Objects in bucket:

- example-a/object/file2.txt

- example-a/object/file3.txt

- example-b/object/file1.txt

- example-b/object/file2.txt

- example-b/object/file3.txt

- example/object/file.txt

## Exercise 8 - List objects by prefix

In this exercise, you will list objects in the S3 bucket by prefix.

In [22]:


def list_all_objects_in_bucket_by_prefix(bucket_name: str, prefix: str) -> list[str]:
    """
    List objects in an S3 bucket by prefix.

    :param bucket_name: Name of the bucket to list objects from
    :param prefix: Prefix to filter objects by
    :return: List of object keys
    """



ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied

## Exercise 9 - Delete a bucket, no matter what!

In this exercise, you will delete the S3 bucket. Your bucket may have objects in it. Does that matter?

Write a function to delete your bucket. 

***Be careful to point it at the right bucket using the right
AWS credentials--or you might delete the wrong bucket!***

In [5]:
import boto3
from botocore.exceptions import ClientError

try:
    from mypy_boto3_s3.type_defs import EmptyResponseMetadataTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def delete_all_objects_in_bucket(bucket_name: str) -> None:
    """
    Delete all objects in an S3 bucket.

    If the bucket does not exist, no error is raised.

    :param bucket_name: Name of the bucket to delete objects from
    """



def delete_bucket(bucket_name: str) -> Optional["EmptyResponseMetadataTypeDef"]:
    """
    :description: Delete an S3 bucket, including all its objects.
        If the bucket does not exist, no error is raised.
    :param: bucket_name (str) Name of the bucket to delete
    :return: Response from the delete_bucket call or None if there is no bucket.
    """


NameError: name 'list_all_object_keys_in_bucket' is not defined

## (Optional) Bonus Exercise #1 - Recursively upload a local directory to S3

In [ ]:
from pathlib import Path
from typing import Generator


def recursively_upload_dir_to_bucket(
    local_dir_fpath: str | Path,
    bucket_name: str,
    target_root_prefix_in_bucket: str = "",
):
    """
    Recurse through a local directory and upload all files to S3 under a target prefix.

    The object keys within the bucket should be the relative paths of the files within the local directory.

    Example:

    path/to/local_dir/
    ├── file1.txt
    ├── file2.txt
    └── subdir
        └── file3.txt

    Would be uploaded to

    s3://bucket-name/target_root_prefix_in_bucket/
    ├── file1.txt
    ├── file2.txt
    └── subdir/
        └── file3.txt
    """
    # fill this out...


# create a test dir locally with sample files
test_dir = Path("test_dir")
test_dir.mkdir(parents=True, exist_ok=True)
(test_dir / "file1.txt").write_text("This is file 1.")
(test_dir / "file2.txt").write_text("This is file 2.")
(test_dir / "subdir").mkdir(parents=True, exist_ok=True)
(test_dir / "subdir" / "file3.txt").write_text("This is file 3.")

# clean up the bucket
delete_bucket(BUCKET_NAME)
create_bucket(bucket_name=BUCKET_NAME)

recursively_upload_dir_to_bucket(
    bucket_name=BUCKET_NAME,
    local_dir_fpath=test_dir,
    target_root_prefix_in_bucket="test-root-dir/",
)

# List all objects in the bucket at the test target root
objects = list_all_objects_in_bucket_by_prefix(
    bucket_name=BUCKET_NAME,
    prefix="test-root-dir/",
)
print(objects)

## (Optional) Bonus Exercise #2 - Rename a "folder" in an S3 bucket

S3 is a key-value store for blobs of bytes. There is no way to "rename" a "folder" in S3
without changing the key names of each object in the "folder".

To really feel the weight and implications of this fact, go through this exercise 🤣

Ultimately, you have to copy each object one at a time, and delete the old object. For large buckets, e.g.
data lakes with millions of files, this is a slow process.

In [ ]:
def rename_folder_in_bucket(
    bucket_name: str,
    old_folder_prefix: str,
    new_folder_prefix: str,
):
    """
    Rename a "folder" in an S3 bucket.

    Example:

    Given the following structure in S3:

    s3://bucket-name/<old_folder_prefix>/
    ├── file1.txt
    ├── file2.txt
    └── subdir/
        └── file3.txt

    After renaming

    s3://bucket-name/<new_folder_prefix>/
    ├── file1.txt
    ├── file2.txt
    └── subdir/
        └── file3.txt

    :param bucket_name: Name of the S3 bucket
    :param source_folder: Source "folder" path in the bucket
    :param destination_folder: Destination "folder" path in the bucket
    """
    # fill this out...


def move_object_in_bucket(bucket_name: str, source_key: str, destination_key: str):
    """
    Move an object within an S3 bucket by copying to the new key and deleting the old key.

    :param bucket_name: Name of the S3 bucket
    :param source_key: Source key of the object to move
    :param destination_key: Destination key of the object
    """
    # fill this out...


"""Test the rename_folder_in_bucket function."""


def upload_file_to_bucket(
    local_fpath: str | Path,
    bucket_name: str,
    target_key_in_bucket: str,
):
    """
    Upload a file to an S3 bucket.

    :param local_fpath: Local file path to upload
    :param bucket_name: Name of the bucket to upload the file to
    :param target_key_in_bucket: Key to upload the file to in the bucket
    """
    local_fpath = Path(local_fpath)
    with open(local_fpath, "rb") as file:
        S3_CLIENT.put_object(Bucket=bucket_name, Key=target_key_in_bucket, Body=file)


# Create test objects in the source folder
test_source_folder = "nested/source-folder/"
test_dest_folder = "nested/destination-folder/"

create_bucket(bucket_name=BUCKET_NAME)

# Upload test files to source folder
upload_file_to_bucket(
    local_fpath="test_dir/file1.txt", bucket_name=BUCKET_NAME, target_key_in_bucket=test_source_folder + "file1.txt"
)
upload_file_to_bucket(
    local_fpath="test_dir/file2.txt", bucket_name=BUCKET_NAME, target_key_in_bucket=test_source_folder + "file2.txt"
)
upload_file_to_bucket(
    local_fpath="test_dir/subdir/file3.txt",
    bucket_name=BUCKET_NAME,
    target_key_in_bucket=test_source_folder + "subdir/file3.txt",
)

# Rename the source folder to the destination folder
rename_folder_in_bucket(
    bucket_name=BUCKET_NAME,
    old_folder_prefix=test_source_folder,
    new_folder_prefix=test_dest_folder,
)

# List all objects in the destination folder
objects = list_all_objects_in_bucket_by_prefix(bucket_name=BUCKET_NAME, prefix=test_dest_folder)
print(objects)